In [1]:
import pandas as pd

In [2]:
query = (
    '''
    SELECT
  *,
  YEAR(contract_date)-TOP AS lasting_year
FROM (
  SELECT
    UPPER(property_name) AS upper_name,
    TOP,
    district,
    units,
    property_type,
    LOWER(tenure) AS lower_tenure,
    FROM
    [drq-machine-learning:enbloc.property_list] )AS property_list
LEFT JOIN (
  SELECT
    contract_date,
    project_name,
    1 AS is_enbloc
  FROM
    [drq-machine-learning:enbloc.enbloc]) AS enbloc_list
ON
  property_list.upper_name = enbloc_list.project_name
  
WHERE 
property_list.property_type!='Office'
    '''
)
df = pd.read_gbq(
    query,
    project_id='drq-machine-learning',
    private_key='/Users/zhouwanjie/Documents/Projects/enbloc/drq-machine-learning-0a5d30a93870.json',
    dialect='legacy'
)



Requesting query... ok.
Job ID: bf14161c-1c64-40ea-b738-edaf4146630a
Query running...
Query done.
Cache hit.

Retrieving results...
Got 2137 rows.

Total time taken 3.68 s.
Finished at 2018-04-15 10:20:34.


In [3]:
df.head()

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year
0,STARLIGHT ROAD,None,D8 - Little India,None,Apartment,9999 years from 01/01/1957,None,None,None,None
1,CAMBRIDGE ROAD,None,D8 - Little India,None,Apartment,999999 years,None,None,None,None
2,JALAN SENANG,None,"D14 - Geylang, Eunos",None,Apartment,999 years from 01/01/1967,None,None,None,None
3,GEK LIM MANSIONS,None,"D14 - Geylang, Eunos",None,Apartment,n.a,None,None,None,None
4,LORONG 24 GEYLANG,None,"D14 - Geylang, Eunos",None,Apartment,9999 years from 01/01/1962,None,None,None,None


In [4]:
df['property_list_lower_tenure'].unique()

array([u'9999 years from 01/01/1957', u'999999 years',
       u'999 years from 01/01/1967', u'n.a',
       u'9999 years from 01/01/1962', u'110 years from 01/11/2017',
       u'999 years', u'99 years from 01/03/1977',
       u'99 years from 25/03/1968', u'99 years from 04/08/1969',
       u'99 years from 02/06/1970', u'99 years from 18/01/1972',
       u'99 years from 03/09/1968', u'99 years from 01/01/1969',
       u'99 years from 30/08/1976', u'99 years from 01/11/1975',
       u'999 years from 07/06/1884', u'99 years from 18/09/1979',
       u'99 years from 16/06/1980', u'99 years from 15/12/1980',
       u'999 years from 25/01/1827', None, u'9999 years from 02/06/1995',
       u'999 years from 29/05/1885', u'99 years from 08/09/1992',
       u'99 years from 01/01/1995', u'999 years from 02/06/1882',
       u'99 years from 17/11/1993', u'99 years from 02/10/1992',
       u'leasehold/99 years', u'929 years from 01/01/1953',
       u'99 years from 13/02/1995', u'99 years from 06/11/19

In [5]:
df = df[df['property_list_lower_tenure'].notnull()]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2135 entries, 0 to 2136
Data columns (total 10 columns):
property_list_upper_name       2135 non-null object
property_list_TOP              2040 non-null object
property_list_district         2135 non-null object
property_list_units            1927 non-null object
property_list_property_type    2135 non-null object
property_list_lower_tenure     2135 non-null object
enbloc_list_contract_date      318 non-null object
enbloc_list_project_name       318 non-null object
enbloc_list_is_enbloc          318 non-null object
lasting_year                   281 non-null object
dtypes: object(10)
memory usage: 183.5+ KB


In [7]:
df=df[df['property_list_lower_tenure']!='n.a']

In [8]:
df.head()

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year
0,STARLIGHT ROAD,None,D8 - Little India,None,Apartment,9999 years from 01/01/1957,None,None,None,None
1,CAMBRIDGE ROAD,None,D8 - Little India,None,Apartment,999999 years,None,None,None,None
2,JALAN SENANG,None,"D14 - Geylang, Eunos",None,Apartment,999 years from 01/01/1967,None,None,None,None
4,LORONG 24 GEYLANG,None,"D14 - Geylang, Eunos",None,Apartment,9999 years from 01/01/1962,None,None,None,None
8,PARKSUITES,1900,"D10 - Bukit Timah, Holland Rd, Tanglin",None,Apartment,110 years from 01/11/2017,None,None,None,None


In [11]:
df['use_year']=df['property_list_lower_tenure'].str.extract('(\d+)(.*)year',expand=False)
df.info()

ValueError: Wrong number of items passed 2, placement implies 1

In [16]:
df['use_year'] = df['property_list_lower_tenure'].apply(lambda x: x.split(' ')[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2126 non-null object
property_list_TOP              2036 non-null object
property_list_district         2126 non-null object
property_list_units            1927 non-null object
property_list_property_type    2126 non-null object
property_list_lower_tenure     2126 non-null object
enbloc_list_contract_date      318 non-null object
enbloc_list_project_name       318 non-null object
enbloc_list_is_enbloc          318 non-null object
lasting_year                   281 non-null object
use_year                       2126 non-null object
dtypes: object(11)
memory usage: 279.3+ KB


In [17]:
df['use_year']=df['use_year'][df['use_year'].notnull()].astype(str)
df['use_year']=df['use_year'].fillna('999')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2126 non-null object
property_list_TOP              2036 non-null object
property_list_district         2126 non-null object
property_list_units            1927 non-null object
property_list_property_type    2126 non-null object
property_list_lower_tenure     2126 non-null object
enbloc_list_contract_date      318 non-null object
enbloc_list_project_name       318 non-null object
enbloc_list_is_enbloc          318 non-null object
lasting_year                   281 non-null object
use_year                       2126 non-null object
dtypes: object(11)
memory usage: 279.3+ KB


In [18]:
df['use_year'].unique()

array(['9999', '999999', '999', '110', '99', 'leasehold/99', '929', '947',
       '946', '956', '60', 'freehold', '101', '100', '103', '99-year',
       'f', 'l99', 'l999', 'mixed'], dtype=object)

In [285]:
df['property_list_units'][df['use_year']<150]=df['property_list_units'][df['use_year']<150].fillna(df['property_list_units'][df['use_year']<150].mean())
df['property_list_units'][df['use_year']>150]=df['property_list_units'][df['use_year']>150].fillna(df['property_list_units'][df['use_year']>150].mean())

/Users/zhouwanjie/virtualenvironment/enbloc/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/zhouwanjie/virtualenvironment/enbloc/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [286]:
df.head(20)

,property_list_upper_name,property_list_TOP,property_list_district,property_list_units,property_list_property_type,property_list_lower_tenure,enbloc_list_contract_date,enbloc_list_project_name,enbloc_list_is_enbloc,lasting_year,use_year
0,STARLIGHT ROAD,None,D8 - Little India,87.6523,Apartment,9999 years from 01/01/1957,None,None,None,None,9999.0
1,CAMBRIDGE ROAD,None,D8 - Little India,87.6523,Apartment,999999 years,None,None,None,None,999999.0
2,JALAN SENANG,None,"D14 - Geylang, Eunos",87.6523,Apartment,999 years from 01/01/1967,None,None,None,None,999.0
4,LORONG 24 GEYLANG,None,"D14 - Geylang, Eunos",87.6523,Apartment,9999 years from 01/01/1962,None,None,None,None,9999.0
8,PARKSUITES,1900,"D10 - Bukit Timah, Holland Rd, Tanglin",422.441,Apartment,110 years from 01/11/2017,None,None,None,None,110.0
9,TEACHERS' HOUSING ESTATE,1968,"D26 - Upper Thomson, Springleaf",87.6523,Apartment,999 years,None,None,None,None,999.0
10,LAKESIDE APARTMENTS,1970,D22 - Jurong,422.441,Apartment,99 years from 01/03/1977,None,None,None,None,99.0
11,PEOPLE'S PARK COMPLEX,1972,"D1 - Temasek Blvd, Raffles Link",422.441,Apartment,99 years from 25/03/1968,None,None,None,None,99.0
12,GOLDEN MILE COMPLEX,1974,"D7 - Middle Road, Golden Mile",422.441,Apartment,99 years from 04/08/1969,None,None,None,None,99.0
13,INTERNATIONAL PLAZA,1976,"D2 - Anson, Tanjong Pagar",422.441,Apartment,99 years from 02/06/1970,None,None,None,None,99.0


In [287]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2126 non-null object
property_list_TOP              2036 non-null object
property_list_district         2126 non-null object
property_list_units            2126 non-null object
property_list_property_type    2126 non-null object
property_list_lower_tenure     2126 non-null object
enbloc_list_contract_date      318 non-null object
enbloc_list_project_name       318 non-null object
enbloc_list_is_enbloc          318 non-null object
lasting_year                   281 non-null object
use_year                       2126 non-null float64
dtypes: float64(1), object(10)
memory usage: 279.3+ KB


In [288]:
df['property_list_TOP']=df['property_list_TOP'].fillna(df['property_list_TOP'].median())

In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 0 to 2136
Data columns (total 11 columns):
property_list_upper_name       2126 non-null object
property_list_TOP              2126 non-null float64
property_list_district         2126 non-null object
property_list_units            2126 non-null object
property_list_property_type    2126 non-null object
property_list_lower_tenure     2126 non-null object
enbloc_list_contract_date      318 non-null object
enbloc_list_project_name       318 non-null object
enbloc_list_is_enbloc          318 non-null object
lasting_year                   281 non-null object
use_year                       2126 non-null float64
dtypes: float64(2), object(9)
memory usage: 279.3+ KB


In [ ]:
df['enbloc_licontact_date']=df['property_list_TOP'].fillna(df['property_list_TOP'].median())